In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# 학습파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx+1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx-1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [3]:
corpus = read_file('./train.txt')

In [4]:
corpus

[[('1', '한편', 'NNG', 'O'),
  ('1', ',', 'SP', 'O'),
  ('2', 'AFC', 'SL', 'O'),
  ('2', '챔피언스', 'NNG', 'O'),
  ('2', '리그', 'NNG', 'O'),
  ('3', 'E', 'SL', 'B_OG'),
  ('3', '조', 'NNG', 'I'),
  ('3', '에', 'JKB', 'O'),
  ('4', '속하', 'VV', 'O'),
  ('4', 'ㄴ', 'ETM', 'O'),
  ('5', '포항', 'NNP', 'O'),
  ('6', '역시', 'MAJ', 'O'),
  ('7', '대회', 'NNG', 'O'),
  ('8', '8강', 'NNG', 'O'),
  ('9', '진출', 'NNG', 'O'),
  ('9', '이', 'JKS', 'O'),
  ('10', '불투명', 'NNG', 'O'),
  ('10', '하', 'VV', 'O'),
  ('10', '다', 'EC', 'O'),
  ('11', '.', 'SF', 'O')],
 [('1', '2003', 'SN', 'B_DT'),
  ('1', '년', 'NNB', 'I'),
  ('2', '6', 'SN', 'I'),
  ('2', '월', 'NNB', 'I'),
  ('3', '14', 'SN', 'I'),
  ('3', '일', 'NNB', 'I'),
  ('4', '사직', 'NNG', 'O'),
  ('5', '두산', 'NNP', 'O'),
  ('5', '전', 'NNG', 'O'),
  ('6', '이후', 'NNG', 'O'),
  ('7', '박명환', 'NNP', 'B_PS'),
  ('7', '에게', 'JKB', 'O'),
  ('8', '당하', 'VV', 'O'),
  ('8', '았', 'EP', 'O'),
  ('8', '던', 'ETM', 'O'),
  ('9', '10', 'SN', 'O'),
  ('9', '연패', 'NNG', 'O'),
  ('10', 

In [5]:
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
  
    sentences.append(sentence)
    tags.append(bio_tag)

print('샘플 크기:', len(sentences))
print('0번째 샘플 문장 시퀀스:', sentences[0])
print('0번째 샘플 bio 태그:', tags[0])
print('샘플 문장 시퀀스 최대 길이:', max(len(l) for l in sentences))
print('샘플 문장 시퀀스 평균 길이:', (sum(map(len, sentences))/len(sentences)))

샘플 크기: 3555
0번째 샘플 문장 시퀀스: ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그: ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이: 168
샘플 문장 시퀀스 평균 길이: 34.03909985935302


In [6]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') # 첫번째 인덱스에는 OOV 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다
tag_tokenizer.fit_on_texts(tags)

In [7]:
# 단어사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index)+1
tag_size = len(tag_tokenizer.word_index)+1
print('BIO 태그 사전 크기:', tag_size)
print('단어 사전 크기:', vocab_size)

BIO 태그 사전 크기: 8
단어 사전 크기: 13834


In [8]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

In [10]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

In [11]:
# 학습, 테스트 데이터 8:2 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [12]:
# 출력데이터를 원핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

In [13]:
print('학습 샘플 시퀀스 형상:', x_train.shape)
print('테스트 샘플 레이블 형상:', y_test.shape)

학습 샘플 시퀀스 형상: (2844, 40)
테스트 샘플 레이블 형상: (711, 40, 8)


In [14]:
# 모델정의
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [15]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.5, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
with tf.device("/device:GPU:0"):
    model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
23/23 [==============================] - 21s 633ms/step - loss: 0.4941 - accuracy: 0.8377
Epoch 2/10
23/23 [==============================] - 15s 661ms/step - loss: 0.2186 - accuracy: 0.9027
Epoch 3/10
23/23 [==============================] - 14s 627ms/step - loss: 0.1431 - accuracy: 0.9303
Epoch 4/10
23/23 [==============================] - 15s 649ms/step - loss: 0.1034 - accuracy: 0.9507
Epoch 5/10
23/23 [==============================] - 15s 634ms/step - loss: 0.0749 - accuracy: 0.9665
Epoch 6/10
23/23 [==============================] - 14s 626ms/step - loss: 0.0529 - accuracy: 0.9769
Epoch 7/10
23/23 [==============================] - 15s 636ms/step - loss: 0.0412 - accuracy: 0.9822
Epoch 8/10
23/23 [==============================] - 14s 627ms/step - loss: 0.0316 - accuracy: 0.9856
Epoch 9/10
23/23 [==============================] - 15s 637ms/step - loss: 0.0264 - accuracy: 0.9879
Epoch 10/10
23/23 [==============================] - 15s 648ms/step - loss: 0.0219 - accura

In [16]:
model.evaluate(x_test, y_test)[1]

23/23 [==============================] - 3s 98ms/step - loss: 0.2024 - accuracy: 0.9373


0.9372843503952026

In [17]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
    for pred in sequence:
        pred_index = np.argmax(pred)
        temp.append(index_to_ner[pred_index].replace("PAD", "O"))
    result.append(temp)
    return result

In [18]:
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER

23/23 [==============================] - 3s 98ms/step


In [20]:
# F1 score 계산위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print('F1 score:', f1_score(test_tags, pred_tags))

              precision    recall  f1-score   support

           _       0.00      0.00      0.00         2
         _DT       0.00      0.00      0.00         1
         _LC       0.50      1.00      0.67         1
         _OG       0.00      0.00      0.00         1

   micro avg       0.17      0.20      0.18         5
   macro avg       0.12      0.25      0.17         5
weighted avg       0.10      0.20      0.13         5

F1 score: 0.1818181818181818


C:\Users\Main\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\Main\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\Main\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [21]:
# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 아이폰에 도전장 내밀다.'.split()
new_x = []

for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])

print('새로운 유형의 시퀀스:', new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0, maxlen=max_len)

새로운 유형의 시퀀스: [531, 307, 1476, 286, 1507, 1, 6766, 1]


In [22]:
# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1)

for w, pred in zip(new_sentence, p[0]):
    print(w, index_to_ner[pred])

1/1 [==============================] - 0s 125ms/step
삼성전자 B_OG
출시 O
스마트폰 O
오늘 B_DT
애플 B_OG
아이폰에 O
도전장 O
내밀다. O
